## Importación de librerias

In [36]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from googletrans import Translator
from translate import Translator
import translate


## Importacion de opciones para Selenium

In [6]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

C:\Users\carlo\AppData\Local\Temp\ipykernel_2544\4076755805.py:6: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  opciones.headless=False    # si True, no aperece la ventana (headless=no visible)


## Importo el csv descargado desde Kaggle sobre razas de perro

In [7]:
perros = pd.read_csv(r'..\dog_breeds.csv',encoding='ISO-8859-1')

### Reviso si el data frame tiene valores nulos o filas repetidas.

In [8]:
perros.isnull().sum()

Breed                     0
Country of Origin         0
Fur Color                 0
Height (in)               0
Color of Eyes             0
Longevity (yrs)           0
Character Traits          0
Common Health Problems    0
dtype: int64

In [9]:
perros.duplicated().sum()

0

# Web Scraping. Parte 1

In [10]:
PATH = ChromeDriverManager().install()    # instala driver de chrome

driver = webdriver.Chrome(PATH)      # abre una ventana de chrome

url = 'https://www.thekennelclub.org.uk/search/breeds-a-to-z/'

driver.get(url)

# Para que pinche en el boton de accept de las cookies y salgan todos los registros.
# En caso de no ponerlo, solo aparecen los primeros registros que le da tiempo a cargar antes de que salte el mensaje de cookies.
driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()

web_perros = driver.find_elements(By.CLASS_NAME,'m-breed-card.m-breed-card--padded')
razas_perros = []

# Iterar por todo el archivo de selenium para que lo convierta en texto.
for dog in web_perros:
    dog = dog.text.strip().split('\n')
    razas_perros.append(dog)

C:\Users\carlo\AppData\Local\Temp\ipykernel_2544\4012704364.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)      # abre una ventana de chrome


In [11]:
# Paso los datos que tengo en texto a un DataFrame
web_perros = pd.DataFrame(razas_perros)
web_perros.head(1)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,TOY,Affenpinscher,Breed characteristics,SIZE,Small,EXERCISE,Up to 1 hour per day,SIZE OF HOME,Flat/ Apartment,GROOMING,...,SHEDS,Yes,LIFESPAN,Over 12 years,VULNERABLE NATIVE BREED,No,TOWN OR COUNTRY,Either,SIZE OF GARDEN,Small/ medium garden


In [12]:
# Elimino las columnas que no me valen
web_perros = web_perros.drop([0, 2, 3, 5, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 19, 21], axis=1)

In [13]:
web_perros.head(1)

,1,4,6,10,12,20,22
0,Affenpinscher,Small,Up to 1 hour per day,More than once a week,Medium,Either,Small/ medium garden


In [14]:
# Cambio los nombres de las columnas para que sea mas visual.
web_perros = web_perros.rename(columns={1: 'Breed', 4: 'Size', 6: 'Exercise', 10: 'Grooming', 12: 'Coat_lenght'
                                      , 20: 'Town_country', 22: 'Size_garden'})

In [15]:
web_perros.head(1)

,Breed,Size,Exercise,Grooming,Coat_lenght,Town_country,Size_garden
0,Affenpinscher,Small,Up to 1 hour per day,More than once a week,Medium,Either,Small/ medium garden


In [16]:
# Hago un left join del nuevo data frame con el descargado de Kaggel. Se añaden a la derecha
# las columnas de la nueva web
perros = perros.merge(web_perros, on = 'Breed', how='left')

# Web scraping. Parte 2

Saco 6 DataFrames de las 5 webs distintas

Junto todo en un unico DataFrame

In [17]:
PATH = ChromeDriverManager().install()    # instala driver de chrome

driver = webdriver.Chrome(PATH)      # abre una ventana de chrome

url1 = 'https://www.expertoanimal.com/razas-de-perros.html'

driver.get(url1)

driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()
web_perros1 = driver.find_elements(By.CLASS_NAME,'resultado.link')

razas_perros1 = []
for dog in web_perros1:
    dog = dog.text.strip().split('\n')
    razas_perros1.append(dog)

driver.quit()

C:\Users\carlo\AppData\Local\Temp\ipykernel_2544\1404346877.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)      # abre una ventana de chrome


In [18]:
PATH = ChromeDriverManager().install()    # instala driver de chrome

driver = webdriver.Chrome(PATH)      # abre una ventana de chrome

url2 = 'https://www.expertoanimal.com/razas-de-perros_2.html'
driver.get(url2)
driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()
web_perros2 = driver.find_elements(By.CLASS_NAME,'resultado.link')

razas_perros2 = []
for dog in web_perros2:
    dog = dog.text.strip().split('\n')
    razas_perros2.append(dog)
    
driver.quit()

C:\Users\carlo\AppData\Local\Temp\ipykernel_2544\525473817.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)      # abre una ventana de chrome


In [20]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()
url3 = 'https://www.expertoanimal.com/razas-de-perros_3.html'
driver.get(url3)

web_perros3 = driver.find_elements(By.CLASS_NAME,'resultado.link')
# web_perros3 = driver.find_elements(By.XPATH,'/html/body/div/div[3]/div[1]/div[2]/div[1]/div/div[3]/a')


razas_perros3 = []
for dog in web_perros1:
    dog = dog.text.strip().split('\n')
    razas_perros3.append(dog)

MaxRetryError: HTTPConnectionPool(host='localhost', port=51854): Max retries exceeded with url: /session/29128a3873f59884c42516d93b4a8cdf/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022B205A30D0>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

In [21]:
PATH = ChromeDriverManager().install()    # instala driver de chrome

driver = webdriver.Chrome(PATH)      # abre una ventana de chromedriver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()


url4 = 'https://www.expertoanimal.com/razas-de-perros_4.html'
driver.get(url4)

driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()

web_perros4 = driver.find_elements(By.CLASS_NAME,'resultado.link')

razas_perros4 = []
for dog in web_perros4:
    dog = dog.text.strip().split('\n')
    razas_perros4.append(dog)
    
driver.quit()

C:\Users\carlo\AppData\Local\Temp\ipykernel_2544\4252638684.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)      # abre una ventana de chromedriver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()


In [22]:
PATH = ChromeDriverManager().install()    # instala driver de chrome

driver = webdriver.Chrome(PATH)      # abre una ventana de chromedriver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()


url5 = 'https://www.expertoanimal.com/razas-de-perros_5.html'
driver.get(url5)

driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()

web_perros5 = driver.find_elements(By.CLASS_NAME,'resultado.link')

razas_perros5 = []
for dog in web_perros5:
    dog = dog.text.strip().split('\n')
    razas_perros5.append(dog)
    
driver.quit()

C:\Users\carlo\AppData\Local\Temp\ipykernel_2544\516894070.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)      # abre una ventana de chromedriver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()


In [23]:
PATH = ChromeDriverManager().install()    # instala driver de chrome

driver = webdriver.Chrome(PATH)      # abre una ventana de chromedriver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()


url6 = 'https://www.expertoanimal.com/razas-de-perros_6.html'
driver.get(url6)

driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()

web_perros6 = driver.find_elements(By.CLASS_NAME,'resultado.link')

razas_perros6 = []
for dog in web_perros6:
    dog = dog.text.strip().split('\n')
    razas_perros6.append(dog)
    
driver.quit()

C:\Users\carlo\AppData\Local\Temp\ipykernel_2544\2813983976.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)      # abre una ventana de chromedriver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()


In [24]:
# Convierto todos los datos extraidos en diferentes DataFrames
razas_perros_df1 = pd.DataFrame(razas_perros1)
razas_perros_df1 = razas_perros_df1.rename(columns={0: 'Breed', 1: 'informacion'})

razas_perros_df2 = pd.DataFrame(razas_perros2)
razas_perros_df2 = razas_perros_df2.rename(columns={0: 'Breed', 1: 'informacion'})

razas_perros_df4 = pd.DataFrame(razas_perros4)
razas_perros_df4 = razas_perros_df4.rename(columns={0: 'Breed', 1: 'informacion'})

razas_perros_df5 = pd.DataFrame(razas_perros5)
razas_perros_df5 = razas_perros_df5.rename(columns={0: 'Breed', 1: 'informacion'})

razas_perros_df6 = pd.DataFrame(razas_perros6)
razas_perros_df6 = razas_perros_df6.rename(columns={0: 'Breed', 1: 'informacion'})

In [80]:
# Junto todos los DataFrames
razas_perros_df = pd.concat([razas_perros_df1, razas_perros_df2, razas_perros_df4, razas_perros_df5, razas_perros_df6])
razas_perros_df

,Breed,informacion
0,Jagd terrier o terrier alemán,"Como su nombre permite suponer, el jagd terrie..."
1,Chihuahua,El perro chihuahua o chihuahueño es una raza d...
2,Perdiguero frisón,"El perdiguero frisón, conocido como Stabyhoun ..."
3,Perdiguero alemán,"Como su nombre indica, el perdiguero alemán (d..."
4,Shih tzu,El shih tzu es una de las razas de perros de c...
...,...,...
20,Fox terrier de pelo liso,El fox terrier de pelo liso es un perro activo...
21,Lhasa apso,"El Lhasa apso, o lhaso apso, es un perro peque..."
22,Papillón,En esta ficha de raza de ExpertoAnimal hablare...
23,Galgo Afgano,El galgo afgano o lebrel afgano es un perro qu...


In [81]:
# compruebo si hay nulos
razas_perros_df.isnull().sum()

Breed          0
informacion    0
dtype: int64

In [82]:
# Elimino las filas que tienen nulos.
razas_perros_df = razas_perros_df.dropna(how='any')
razas_perros_df

,Breed,informacion
0,Jagd terrier o terrier alemán,"Como su nombre permite suponer, el jagd terrie..."
1,Chihuahua,El perro chihuahua o chihuahueño es una raza d...
2,Perdiguero frisón,"El perdiguero frisón, conocido como Stabyhoun ..."
3,Perdiguero alemán,"Como su nombre indica, el perdiguero alemán (d..."
4,Shih tzu,El shih tzu es una de las razas de perros de c...
...,...,...
20,Fox terrier de pelo liso,El fox terrier de pelo liso es un perro activo...
21,Lhasa apso,"El Lhasa apso, o lhaso apso, es un perro peque..."
22,Papillón,En esta ficha de raza de ExpertoAnimal hablare...
23,Galgo Afgano,El galgo afgano o lebrel afgano es un perro qu...


La informacion extraida de esta ultima web estaba en Español, por lo que no puedo hacer un merge ya que las razas no son iguales.

Hago una funcion para traducir strings

Le paso esta funcion a todo el DataFrame

Mi primera opcion para traducir el texto de español a ingles fue con esta funcion pero me convierte todos los valores a nulos

from mtranslate import translate

def translate_text(text):
    translation = translate(text, 'en') # Idioma destino: inglés
    return translation

razas_perros_df['Breed'] = razas_perros_df['Breed'].apply(translate_text)

Mi segunda opcion tuvo la misma respuesta.

import time
from googletrans import Translator

def translate_text(text):
    translator = Translator(service_urls=['translate.google.com'])
    translation = translator.translate(text, src='es', dest='en', timeout=60)
    return translation.text

razas_perros_df['Breed'] = razas_perros_df['Breed'].apply(translate_text)

In [83]:
# Tercera opcion para traducir de español a ingles. Esta si me dejaba.
def traducir_strings(texto):

    translator = translate.Translator(from_lang="es", to_lang="en")
    texto_traducido = translator.translate(texto)
    return texto_traducido

In [84]:
razas_perros_df['Breed'] = razas_perros_df['Breed'].apply(traducir_strings)

In [86]:
razas_perros_df['informacion'] = razas_perros_df['informacion'].apply(traducir_strings)

In [90]:
# Hago un left join del nuevo DataFrame con el original.

perros = perros.merge(razas_perros_df, on = 'Breed', how='left')

In [91]:
# Compruebo nulos
perros.isnull().sum()

Breed                      0
Country of Origin          0
Fur Color                  0
Height (in)                0
Color of Eyes              0
Longevity (yrs)            0
Character Traits           0
Common Health Problems     0
Size                      43
Exercise                  43
Grooming                  43
Coat_lenght               43
Town_country              43
Size_garden               43
informacion               79
dtype: int64

In [92]:
# Relleno nulos con un mensaje de NoInfo
perros = perros.fillna(value='NoInfo')

In [93]:
# Confirmo que no tengo nulos
perros.isna().sum()

Breed                     0
Country of Origin         0
Fur Color                 0
Height (in)               0
Color of Eyes             0
Longevity (yrs)           0
Character Traits          0
Common Health Problems    0
Size                      0
Exercise                  0
Grooming                  0
Coat_lenght               0
Town_country              0
Size_garden               0
informacion               0
dtype: int64

In [94]:
perros.head()

,Breed,Country of Origin,Fur Color,Height (in),Color of Eyes,Longevity (yrs),Character Traits,Common Health Problems,Size,Exercise,Grooming,Coat_lenght,Town_country,Size_garden,informacion
0,Labrador Retriever,Canada,"Yellow, Black, Chocolate",21-24,Brown,10-12,"Loyal, friendly, intelligent, energetic, good-...","Hip dysplasia, obesity, ear infections",NoInfo,NoInfo,NoInfo,NoInfo,NoInfo,NoInfo,NoInfo
1,German Shepherd,Germany,"Black, Tan",22-26,Brown,7-10,"Loyal, intelligent, protective, confident, tra...","Hip dysplasia, elbow dysplasia, pancreatitis",NoInfo,NoInfo,NoInfo,NoInfo,NoInfo,NoInfo,NoInfo
2,Bulldog,England,"White, Red",12-16,Brown,8-10,"Loyal, calm, gentle, brave","Skin allergies, respiratory issues, obesity",Medium,Up to 1 hour per day,Once a week,Short,Either,Small/ medium garden,NoInfo
3,Poodle,France,"White, Black, Brown, Apricot",10-15,"Brown, Blue",12-15,"Intelligent, active, affectionate, hypoallergenic","Hip dysplasia, epilepsy, bladder stones",NoInfo,NoInfo,NoInfo,NoInfo,NoInfo,NoInfo,NoInfo
4,Beagle,England,"White, Tan, Red, Lemon",13-15,Brown,12-15,"Curious, friendly, energetic, good-natured","Ear infections, hip dysplasia, epilepsy",Small,Up to 1 hour per day,Once a week,Short,Either,Small/ medium garden,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...


In [95]:
perros.to_csv(r'..\clean_data\perros.csv', index=True)